In [ ]:
!pip install -U -q sentence-transformers dppy datasets transformers flair

!nvidia-smi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 KB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 81.7 MB/s eta 0:00:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import load_dataset, concatenate_datasets
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from dppy.finite_dpps import FiniteDPP
from scipy.linalg import qr
from numpy import linalg as LA


from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings("ignore")

import altair as alt
alt.renderers.enable('colab')

RendererRegistry.enable('colab')

In [ ]:
mtop = load_dataset("iohadrubin/mtop")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset mtop downloaded and prepared to /root/.cache/huggingface/datasets/iohadrubin___mtop/mtop/1.0.0/4ba6d9db9efaebd4f6504db7e36925632e959f456071b9d7f1b86a85cce52448. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
mtop



DatasetDict({
    train: Dataset({
        features: ['idx', 'intent', 'spans', 'question', 'domain', 'lang', 'logical_form', 'tokenized_question'],
        num_rows: 15667
    })
    validation: Dataset({
        features: ['idx', 'intent', 'spans', 'question', 'domain', 'lang', 'logical_form', 'tokenized_question'],
        num_rows: 2235
    })
    test: Dataset({
        features: ['idx', 'intent', 'spans', 'question', 'domain', 'lang', 'logical_form', 'tokenized_question'],
        num_rows: 4386
    })
})

In [ ]:
mtop['validation'], mtop['test']

(Dataset({
     features: ['idx', 'intent', 'spans', 'question', 'domain', 'lang', 'logical_form', 'tokenized_question'],
     num_rows: 2235
 }), Dataset({
     features: ['idx', 'intent', 'spans', 'question', 'domain', 'lang', 'logical_form', 'tokenized_question'],
     num_rows: 4386
 }))

In [ ]:
# test_dataset = concatenate_datasets([mtop['validation'], mtop['test']])
test_dataset = mtop['test']

In [ ]:
test_dataset

Dataset({
    features: ['idx', 'intent', 'spans', 'question', 'domain', 'lang', 'logical_form', 'tokenized_question'],
    num_rows: 4386
})

In [ ]:
domains = list(np.unique(test_dataset['domain']))

In [ ]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
seed = 42

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
sentences = test_dataset['question']

# vectors = get_bert_embedding_batch(sentences)

In [ ]:
# sentences = test_dataset['question']
vectors = model.encode(sentences)

# Set a seed for reproducibility


vectors


array([[-0.01916886,  0.09027083, -0.0600341 , ..., -0.0205449 ,
         0.01856751,  0.02931697],
       [-0.03582959,  0.01181684, -0.01800052, ...,  0.05906511,
        -0.02521114,  0.03805231],
       [ 0.01684192, -0.00088983, -0.01582805, ...,  0.00019043,
        -0.08045583,  0.01754356],
       ...,
       [ 0.03524666,  0.01417136, -0.03573525, ...,  0.02033618,
        -0.05856695, -0.04539205],
       [-0.02743943, -0.02805242,  0.01485658, ...,  0.04142861,
        -0.07461242, -0.00244304],
       [ 0.00449554,  0.02701857, -0.00381332, ...,  0.04358262,
         0.04643188, -0.01202127]], dtype=float32)

In [ ]:


eig_vecs, _ = qr(vectors)
eigenvals_sentences = LA.eigvals(eig_vecs).astype("float64")

L_rows = (eig_vecs * eigenvals_sentences).dot(eig_vecs.T)  # m x m

In [ ]:
LA.matrix_rank(L_rows)

4386

In [ ]:
# vectors.dot(vectors.T).shape

In [ ]:
# LA.matrix_rank(vectors.dot(vectors.T))

In [ ]:
# L_rows = vectors.dot(vectors.T)  # m x m
# rng = np.random.RandomState(413121)

# dpp_rows = FiniteDPP(kernel_type='likelihood', L=vectors.dot(vectors.T))

dpp_rows = FiniteDPP(kernel_type='likelihood', L=L_rows)




In [ ]:
# sub_rows = dpp_rows.sample_exact(random_state=seed)
seed=0

flag = True

while flag:
  try:
    # sub_rows = dpp_rows.sample_exact_k_dpp(size= 60, random_state=seed)
    sub_rows = dpp_rows.sample_exact(random_state=seed)

    flag = False
  except:
    seed += 1

In [ ]:
len(sub_rows)

1809

In [ ]:
sub_rows

[2458,
 2452,
 3619,
 1610,
 379,
 2497,
 105,
 1511,
 4324,
 3767,
 633,
 3446,
 3494,
 4049,
 929,
 2107,
 3573,
 2847,
 2529,
 2780,
 1000,
 104,
 3171,
 1379,
 1594,
 2632,
 2822,
 848,
 3595,
 1625,
 2733,
 4228,
 2430,
 1522,
 2673,
 2157,
 1967,
 3205,
 4048,
 2081,
 373,
 2477,
 1067,
 177,
 3443,
 1199,
 3402,
 2117,
 1378,
 1355,
 184,
 2301,
 3583,
 333,
 436,
 2244,
 2818,
 2947,
 1319,
 132]

In [ ]:
# dpp_rows.sample_mcmc_k_dpp(size=413, random_state=rng)
# dpp_rows.sample_mcmc('AED', random_state = rng)

In [ ]:
subdataset = test_dataset.select(sub_rows)


In [ ]:
from collections import defaultdict

counts = defaultdict(int)

In [ ]:
for data in subdataset:
    counts[data['domain']] += 1

print(counts)

defaultdict(<class 'int'>, {'reminder': 254, 'people': 151, 'news': 144, 'calling': 245, 'weather': 195, 'event': 80, 'timer': 128, 'recipes': 131, 'alarm': 189, 'music': 155, 'messaging': 137})


In [ ]:
subdataset.to_pandas().to_csv("/content/drive/MyDrive/IndicSemParse/itop_sampled_test_dev.csv")

In [ ]:
subdataset.to_pandas()['domain'].value_counts()

reminder     254
calling      245
weather      195
alarm        189
music        155
people       151
news         144
messaging    137
recipes      131
timer        128
event         80
Name: domain, dtype: int64

In [ ]:
test_dataset.to_pandas()['domain'].value_counts()

reminder     613
calling      613
weather      487
alarm        418
music        369
news         358
people       354
recipes      346
messaging    341
timer        290
event        197
Name: domain, dtype: int64

In [ ]:
import json
from datasets import Dataset, DatasetDict

def create_dataset(dataset, train_lang, test_lang, backtranslated=False):

    if train_lang != "en":
        with open(f"/content/drive/MyDrive/IndicSemParse/data/{dataset}/{train_lang}.json", "r") as f:
            train_data = json.load(f)

    else:
        with open(f"/content/drive/MyDrive/IndicSemParse/data/{dataset}/hi.json", "r") as f:
            train_data = json.load(f)

    if test_lang != "en":
        with open(f"/content/drive/MyDrive/IndicSemParse/data/{dataset}/{test_lang}.json", "r") as f:
            test_data = json.load(f)

    else:
        with open(f"/content/drive/MyDrive/IndicSemParse/data/{dataset}/hi.json", "r") as f:
            test_data = json.load(f)

    train = train_data["train"]

    val = test_data["validation"]

    test = test_data["test"]

    if dataset == "itop":
        for idx, example in enumerate(train):
            if train_lang != "en":
                train[idx]["original_src"] = example["question"]
                train[idx]["translated_src"] = example["postprocessed_translated_question"]
            else:
                train[idx]["src"] = example["question"]
            train[idx]["trg"] = example["logical_form"]

        for idx, example in enumerate(val):
            if test_lang != "en":
                if backtranslated:
                    val[idx]["orig"] = example["question"]
                    val[idx]["src"] = example["backtranslated_post_processed_questions"]
                else:
                    val[idx]["orig"] = example["question"]
                    val[idx]["src"] = example["postprocessed_translated_question"]
            else:
                val[idx]["src"] = example["question"]
            val[idx]["trg"] = example["logical_form"]

        for idx, example in enumerate(test):
            if test_lang != "en":
                if backtranslated:
                    test[idx]["orig"] = example["question"]
                    test[idx]["src"] = example[
                        "backtranslated_post_processed_questions"
                    ]
                else:
                    test[idx]["orig"] = example["question"]
                    test[idx]["src"] = example["postprocessed_translated_question"]
            else:
                test[idx]["src"] = example["question"]
            test[idx]["trg"] = example["logical_form"]

    elif dataset == "indic-TOP":
        for idx, example in enumerate(train):
            if train_lang != "en":
                train[idx]["src"] = example["postprocessed_translated_question"]
            else:
                train[idx]["src"] = example["question"]
            train[idx]["trg"] = example["decoupled logical form"]

        for idx, example in enumerate(val):
            if test_lang != "en":
                if backtranslated:
                    val[idx]["orig"] = example["question"]
                    val[idx]["src"] = example["backtranslated_post_processed_questions"]
                else:
                    val[idx]["orig"] = example["question"]
                    val[idx]["src"] = example["postprocessed_translated_question"]
            else:
                val[idx]["src"] = example["question"]
            val[idx]["trg"] = example["decoupled logical form"]

        for idx, example in enumerate(test):
            if test_lang != "en":
                if backtranslated:
                    test[idx]["orig"] = example["question"]
                    test[idx]["src"] = example[
                        "backtranslated_post_processed_questions"
                    ]
                else:
                    test[idx]["orig"] = example["question"]
                    test[idx]["src"] = example["postprocessed_translated_question"]
            else:
                test[idx]["src"] = example["question"]
            test[idx]["trg"] = example["decoupled logical form"]


    elif dataset == "indic-atis":
        for idx, example in enumerate(train):
            if train_lang != "en":
                train[idx]["src"] = example["translated text"]
            else:
                train[idx]["src"] = example["text"]
            train[idx]["trg"] = example["logical form"]

        for idx, example in enumerate(val):
            if test_lang != "en":
                if backtranslated:
                    val[idx]["orig"] = example["text"]
                    val[idx]["src"] = example["back translated text"]
                else:
                    val[idx]["orig"] = example["text"]
                    val[idx]["src"] = example["translated text"]
            else:
                val[idx]["src"] = example["text"]
            val[idx]["trg"] = example["logical form"]

        for idx, example in enumerate(test):
            if test_lang != "en":
                if backtranslated:
                    test[idx]["orig"] = example["text"]
                    test[idx]["src"] = example["back translated text"]
                else:
                    test[idx]["orig"] = example["text"]
                    test[idx]["src"] = example["translated text"]
            else:
                test[idx]["src"] = example["text"]
            test[idx]["trg"] = example["logical form"]

    train_data = Dataset.from_pandas(pd.DataFrame(data=train))

    val_data = Dataset.from_pandas(pd.DataFrame(data=val))

    test_data = Dataset.from_pandas(pd.DataFrame(data=test))

    dataset = DatasetDict()

    dataset["train"] = train_data

    dataset["val"] = val_data

    dataset["test"] = test_data

    return dataset



In [ ]:
indictop = create_dataset("indic-TOP", "en", "en")

indictop

DatasetDict({
    train: Dataset({
        features: ['original question', 'translated question', 'logical form', 'question', 'postprocessed_translated_question', 'decoupled logical form', 'backtranslated_post_processed_questions', 'src', 'trg'],
        num_rows: 31279
    })
    val: Dataset({
        features: ['original question', 'translated question', 'logical form', 'question', 'postprocessed_translated_question', 'decoupled logical form', 'backtranslated_post_processed_questions', 'src', 'trg'],
        num_rows: 4462
    })
    test: Dataset({
        features: ['original question', 'translated question', 'logical form', 'question', 'postprocessed_translated_question', 'decoupled logical form', 'backtranslated_post_processed_questions', 'src', 'trg'],
        num_rows: 9042
    })
})

In [ ]:
# test_dataset = concatenate_datasets([indictop['val'], indictop['test']])

test_dataset = indictop['test']


test_dataset

Dataset({
    features: ['original question', 'translated question', 'logical form', 'question', 'postprocessed_translated_question', 'decoupled logical form', 'backtranslated_post_processed_questions', 'src', 'trg'],
    num_rows: 9042
})

In [ ]:
sentences = test_dataset['original question']
sentences

['whats there to do this weekend',
 'What is a good restaurant for Tex Mex in Austin',
 'Where can I see the fireworks tonight',
 'Restaurants offering prefixed menus in midtown',
 'Holiday recipes',
 'Where should I go dancing this weekend',
 'What is going on this weekend',
 'are there any santa meetings this weekend',
 'What breakfast locations within 5 miles of me open at 6 am',
 'What can i do with my friends tomorrow night',
 "Who has the latest tennis shoes out for youth today that's close to where I am right now me",
 'Beer tasting parties',
 'Anything fun happening',
 'What time is Mass on Easter Sunday',
 'Concerts by Blake Shelton for this weekend',
 'Concerts by Beyonce for this weekend',
 'List near me vegetarian restaurants that specialize in Quinoa',
 'Santa pictures this weekend',
 'Family parks in Houston',
 'Cheap things to do',
 'Where is Lorde performing this weekend',
 'How cold iwill it be outside tonight',
 'When does Run through the Lights start',
 'know of any 

In [ ]:
vectors = model.encode(sentences)

# Set a seed for reproducibility


vectors


array([[-0.0479308 , -0.03104286, -0.02706314, ...,  0.01039248,
         0.01059171,  0.00219955],
       [-0.03396904,  0.01558107, -0.010593  , ...,  0.00468091,
         0.05731867, -0.00726815],
       [-0.09710886,  0.02066339, -0.01194812, ...,  0.0241541 ,
        -0.00862841, -0.00421765],
       ...,
       [-0.06680109,  0.02794166, -0.02321045, ..., -0.01580125,
        -0.05408265, -0.01296461],
       [-0.01997896,  0.02575457, -0.0488971 , ..., -0.08643141,
        -0.0316636 , -0.01614545],
       [-0.04127202,  0.02028693, -0.0392466 , ..., -0.03554438,
         0.01129051, -0.00849326]], dtype=float32)

In [ ]:

eig_vecs, _ = qr(vectors)
eigenvals_sentences = LA.eigvals(eig_vecs).astype("float64")

L_rows = (eig_vecs * eigenvals_sentences).dot(eig_vecs.T)  # m x m

print(LA.matrix_rank(L_rows))

dpp_rows = FiniteDPP(kernel_type='likelihood', L=L_rows)


seed=0

flag = True

while flag:
  try:
    # sub_rows = dpp_rows.sample_exact_k_dpp(size= 120, random_state=seed)
    sub_rows = dpp_rows.sample_exact(random_state=seed)
    flag = False
  except:
    seed += 1

9042


In [ ]:
len(sub_rows)

4150

In [ ]:
sub_rows

[2965,
 6235,
 1887,
 4996,
 4485,
 5319,
 7553,
 6193,
 7978,
 2544,
 2277,
 1714,
 77,
 5951,
 3384,
 1516,
 2361,
 1663,
 1625,
 2079,
 880,
 4931,
 7563,
 3162,
 8252,
 6194,
 7469,
 8363,
 6506,
 6967,
 676,
 1723,
 2862,
 3052,
 4225,
 870,
 3140,
 668,
 2953,
 5990,
 27,
 2593,
 666,
 6436,
 7611,
 8846,
 5587,
 2054,
 2086,
 4479,
 8422,
 5686,
 1654,
 4679,
 3725,
 7860,
 955,
 1987,
 6871,
 8800,
 7313,
 5169,
 7320,
 3450,
 2189,
 4022,
 2673,
 5835,
 8283,
 6080,
 7089,
 8318,
 6624,
 4144,
 6715,
 8957,
 6100,
 6580,
 1234,
 7390,
 8401,
 757,
 8338,
 6874,
 2305,
 322,
 5368,
 1277,
 7962,
 5819,
 380,
 1775,
 5649,
 4769,
 2646,
 3597,
 2250,
 9027,
 7986,
 6701,
 2249,
 812,
 879,
 7969,
 2043,
 505,
 5506,
 6820,
 8186,
 4390,
 6499,
 432,
 2776,
 797,
 3646,
 1821,
 4522,
 2209,
 1746,
 5445,
 4362,
 3477,
 3814,
 8562,
 5531,
 1842,
 2437,
 2734,
 6440,
 4621,
 1939,
 5257,
 7916,
 1152,
 8973,
 5193,
 690,
 8361,
 5987,
 2360,
 105,
 7485,
 1797,
 8230,
 2396,
 2808

In [ ]:
subdataset = test_dataset.select(sub_rows)


subdataset.to_pandas().to_csv("/content/drive/MyDrive/IndicSemParse/indictop_sampled_test_dev.csv")

In [ ]:
subdataset.to_pandas()

,original question,translated question,logical form,question,postprocessed_translated_question,decoupled logical form,backtranslated_post_processed_questions,src,trg
0,Is there an accident on my home,क्या मेरे घर पर कोई दुर्घटना हुई है?,[IN:GET_INFO_TRAFFIC Is there an accident on [...,क्या मेरे घर पर कोई दुर्घटना हुई है?,क्या मेरे हाउस ऑन कोई दुर्घटना हुई है?,[IN:GET_INFO_TRAFFIC [SL:DESTINATION [IN:GET_L...,Have there been any accidents at my house?,क्या मेरे घर पर कोई दुर्घटना हुई है?,[IN:GET_INFO_TRAFFIC [SL:DESTINATION [IN:GET_L...
1,can you tell me if there is any congestion bet...,क्या आप मुझे बता सकते हैं कि 15 मई शहर और हेलव...,[IN:GET_INFO_TRAFFIC can you tell me if there ...,क्या आप मुझे बता सकते हैं कि 15 मई शहर और हेलव...,क्या आप मुझे बता सकते हैं कि 15 मई सिटी एंड हे...,[IN:GET_INFO_TRAFFIC [SL:SOURCE 15 May city ] ...,Can you tell me that there is still a crowd be...,क्या आप मुझे बता सकते हैं कि 15 मई शहर और हेलव...,[IN:GET_INFO_TRAFFIC [SL:SOURCE 15 May city ] ...
2,whats the party at the playboy mansion on satu...,शनिवार को प्लेबॉय हवेली में पार्टी क्या होती है?,[IN:GET_EVENT whats [SL:CATEGORY_EVENT the par...,शनिवार को प्लेबॉय हवेली में पार्टी क्या होती है?,शनिवार को प्लेबॉय हवेली इन पार्टी क्या होती है?,[IN:GET_EVENT [SL:CATEGORY_EVENT the party ] [...,What is Playboy Haveli in Party on Saturday?,शनिवार को प्लेबॉय हवेली में पार्टी क्या होती है?,[IN:GET_EVENT [SL:CATEGORY_EVENT the party ] [...
3,May I have a traffic update,क्या मुझे ट्रैफिक अपडेट मिल सकता है?,[IN:GET_INFO_TRAFFIC May I have a traffic upda...,क्या मुझे ट्रैफिक अपडेट मिल सकता है?,क्या मुझे ट्रैफिक अपडेट मिल सकता है?,[IN:GET_INFO_TRAFFIC ],Can I get traffic updates?,क्या मुझे ट्रैफिक अपडेट मिल सकता है?,[IN:GET_INFO_TRAFFIC ]
4,are there any wrecks on my route to work,क्या मेरे काम करने के रास्ते में कोई मलबा है?,[IN:GET_INFO_TRAFFIC are there any wrecks on m...,क्या मेरे काम करने के रास्ते में कोई मलबा है?,क्या मेरे काम करने के रास्ते इन कोई मलबा है?,[IN:GET_INFO_TRAFFIC [SL:DESTINATION [IN:GET_L...,Are there any debris on my way to work?,क्या मेरे काम करने के रास्ते में कोई मलबा है?,[IN:GET_INFO_TRAFFIC [SL:DESTINATION [IN:GET_L...
...,...,...,...,...,...,...,...,...,...
4145,When is the traffic worst on riverdale road,रिवरडेल रोड पर यातायात सबसे खराब कब होता है?,[IN:GET_INFO_TRAFFIC When is the traffic worst...,रिवरडेल रोड पर यातायात सबसे खराब कब होता है?,रिवरडेल रोड ऑन यातायात सबसे खराब कब होता है?,[IN:GET_INFO_TRAFFIC [SL:LOCATION riverdale ro...,When is Riverdale Road on traffic worst?,रिवरडेल रोड पर यातायात सबसे खराब कब होता है?,[IN:GET_INFO_TRAFFIC [SL:LOCATION riverdale ro...
4146,Are there any events available next Tuesday,क्या अगले मंगलवार को कोई कार्यक्रम उपलब्ध है?,[IN:GET_EVENT Are there any events available [...,क्या अगले मंगलवार को कोई कार्यक्रम उपलब्ध है?,क्या अगले मंगलवार को कोई कार्यक्रम उपलब्ध है?,[IN:GET_EVENT [SL:DATE_TIME next Tuesday ] ],Is there any programme available on next Tuesday?,क्या अगले मंगलवार को कोई कार्यक्रम उपलब्ध है?,[IN:GET_EVENT [SL:DATE_TIME next Tuesday ] ]
4147,How far is sweden from norway,नॉर्वे से कितनी दूर है?,[IN:GET_DISTANCE How far is [SL:DESTINATION sw...,नॉर्वे से कितनी दूर है?,नॉर्वे से कितनी दूर है?,[IN:GET_DISTANCE [SL:DESTINATION sweden ] [SL:...,How far is it from Norway?,नॉर्वे से कितनी दूर है?,[IN:GET_DISTANCE [SL:DESTINATION sweden ] [SL:...
4148,Anything to do today,आज करने के लिए कुछ भी,[IN:GET_EVENT Anything to do [SL:DATE_TIME tod...,आज करने के लिए कुछ भी,आज करने के लिए कुछ भी,[IN:GET_EVENT [SL:DATE_TIME today ] ],Anything to do today,आज करने के लिए कुछ भी,[IN:GET_EVENT [SL:DATE_TIME today ] ]


In [ ]:
indicatis = create_dataset("indic-atis", "en", "en")

indicatis

DatasetDict({
    train: Dataset({
        features: ['text', 'intent', 'entities', 'translated text', 'back translated text', 'logical form', 'decoupled logical form', 'src', 'trg'],
        num_rows: 4470
    })
    val: Dataset({
        features: ['text', 'intent', 'entities', 'translated text', 'back translated text', 'logical form', 'decoupled logical form', 'src', 'trg'],
        num_rows: 446
    })
    test: Dataset({
        features: ['text', 'intent', 'entities', 'translated text', 'back translated text', 'logical form', 'decoupled logical form', 'src', 'trg'],
        num_rows: 893
    })
})

In [ ]:
# test_dataset = concatenate_datasets([indicatis['val'], indicatis['test']])

test_dataset = indicatis['test']

test_dataset

Dataset({
    features: ['text', 'intent', 'entities', 'translated text', 'back translated text', 'logical form', 'decoupled logical form', 'src', 'trg'],
    num_rows: 893
})

In [ ]:
sentences = test_dataset['text']
sentences

vectors = model.encode(sentences)

# Set a seed for reproducibility


vectors

array([[-1.8956786e-02,  5.5822384e-02, -3.6465179e-02, ...,
         2.2319989e-02,  1.0359873e-02, -5.6639905e-03],
       [-5.4788943e-03,  8.0241859e-02, -2.8604908e-02, ...,
        -2.0322213e-02, -2.6273340e-02, -6.6258553e-03],
       [-2.8171061e-02,  7.0857301e-02, -5.6559607e-02, ...,
        -9.0969270e-03,  1.6570548e-02,  1.3367192e-02],
       ...,
       [-5.2177187e-02,  9.0419777e-02, -5.5350658e-02, ...,
         1.9504979e-02, -3.4354385e-02, -2.2245951e-02],
       [-2.6521737e-02,  1.7564664e-05, -2.7007675e-02, ...,
         3.0037997e-02, -2.8664129e-02, -5.6538833e-03],
       [-2.6254244e-03,  8.6803772e-02, -3.2189671e-02, ...,
        -1.3376297e-02, -4.6055991e-02, -1.5129880e-02]], dtype=float32)

In [ ]:
eig_vecs, _ = qr(vectors)
eigenvals_sentences = LA.eigvals(eig_vecs).astype("float64")

L_rows = (eig_vecs * eigenvals_sentences).dot(eig_vecs.T)  # m x m

print(LA.matrix_rank(L_rows))

dpp_rows = FiniteDPP(kernel_type='likelihood', L=L_rows)


seed=0

flag = True

while flag:
  try:
    # sub_rows = dpp_rows.sample_exact_k_dpp(size= 20, random_state=seed)
    sub_rows = dpp_rows.sample_exact(random_state=seed)
    flag = False
  except:
    seed += 1

893


In [ ]:
len(sub_rows)

166

In [ ]:
sub_rows

[452,
 871,
 351,
 685,
 532,
 779,
 343,
 239,
 797,
 40,
 840,
 535,
 362,
 887,
 57,
 501,
 31,
 547,
 175,
 579,
 872,
 767,
 437,
 626,
 688,
 478,
 507,
 150,
 355,
 144,
 636,
 460,
 433,
 776,
 509,
 389,
 259,
 69,
 402,
 27,
 280,
 490,
 855,
 110,
 604,
 488,
 690,
 146,
 82,
 382,
 708,
 189,
 676,
 282,
 214,
 17,
 384,
 367,
 598,
 70,
 157,
 18,
 118,
 594,
 394,
 316,
 546,
 885,
 740,
 623,
 805,
 37,
 74,
 468,
 344,
 824,
 486,
 536,
 784,
 439,
 769,
 253,
 270,
 326,
 542,
 221,
 608,
 151,
 99,
 764,
 240,
 379,
 558,
 524,
 79,
 768,
 841,
 720,
 279,
 521,
 335,
 550,
 865,
 307,
 606,
 320,
 687,
 318,
 780,
 660,
 669,
 461,
 206,
 41,
 315,
 35,
 622,
 603,
 425,
 678,
 528,
 45,
 143,
 196,
 338,
 59,
 572,
 506,
 575,
 421,
 749,
 441,
 14,
 765,
 599,
 732,
 843,
 58,
 233,
 73,
 409,
 757,
 94,
 465,
 138,
 684,
 745,
 52,
 479,
 238,
 844,
 833,
 574,
 862,
 340,
 787,
 416,
 224,
 829,
 613,
 77,
 759,
 620,
 564,
 635,
 191]

In [ ]:
subdataset = test_dataset.select(sub_rows)


subdataset.to_pandas().to_csv("/content/drive/MyDrive/IndicSemParse/indicatis_sampled_test_dev.csv")

In [ ]:
test_dataset.to_pandas()['intent'].nunique()

20

In [ ]:
test_dataset.to_pandas()['intent'].value_counts()

flight               632
airfare               48
airline               38
ground_service        36
abbreviation          33
capacity              21
airport               18
flight+airfare        12
distance              10
aircraft               9
flight_no              8
ground_fare            7
city                   6
meal                   6
quantity               3
day_name               2
flight_time            1
airfare+flight         1
flight+airline         1
flight_no+airline      1
Name: intent, dtype: int64

In [ ]:
subdataset.to_pandas()['intent'].value_counts()

flight            105
abbreviation       11
airline            11
airfare            10
airport             8
ground_service      5
capacity            4
aircraft            3
flight_no           2
flight+airfare      2
ground_fare         2
distance            2
meal                1
Name: intent, dtype: int64